In [1]:
import pandas as pd
import datetime

In [2]:
df = pd.read_csv('./data/data.csv').set_index('date')
df.index = pd.to_datetime(df.index)
dup_df = pd.read_csv('./data/updated_ratings.csv').set_index('date')

In [3]:
dup_df

,rating,report,problems
date,,,
2019-02-25,high,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Storm Slab']"
2019-02-25,moderate,https://www.sierraavalanchecenter.org/advisory...,['Wind Slab']
2019-02-09,high,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Storm Slab']"
2019-02-09,considerable,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Storm Slab']"
2018-03-04,considerable,https://www.sierraavalanchecenter.org/advisory...,"['Deep Slab', 'Wind Slab', 'Loose Wet']"
2018-03-04,moderate,https://www.sierraavalanchecenter.org/advisory...,"['Wind Slab', 'Deep Slab', 'Loose Wet']"


In [4]:
df.rating.value_counts()

moderate        925
low             537
considerable    256
high             60
na               56
extream           2
Name: rating, dtype: int64

In [5]:
low_df = df[df.rating == 'low']
mod_df = df[df.rating == 'moderate']
con_df = df[df.rating == 'considerable']
hig_df = df[df.rating == 'high']
ext_df = df[df.rating == 'extreame']

In [6]:
window = np.timedelta64(3,'D')

In [7]:
end_date = low_df.sample().index.values[0]
start_date = (end_date - window)

In [8]:
df[(df.index > start_date) & (df.index <= end_date)]

,Snow Depth (cm) Start of Day Values,Precipitation Increment (mm),Air Temperature Average (degC),Wind Direction Average (degree),Wind Speed Average (km/hr),rating,report,problems
date,,,,,,,,
2020-03-10,79.200000,0.000000,1.186667,184.0,3.45,low,https://www.sierraavalanchecenter.org/advisory...,['Loose Wet']
2020-03-11,78.400000,0.333333,2.433333,226.5,2.80,low,https://www.sierraavalanchecenter.org/advisory...,['Loose Wet']
2020-03-12,75.733333,0.000000,3.293333,53.5,5.50,low,https://www.sierraavalanchecenter.org/advisory...,['Loose Wet']


In [9]:
_df = df.drop(['report', 'problems', 'Wind Direction Average (degree)'], axis=1)
values = [0, 1, 2, 3, 4, 5]
conditions = (
    df['rating'] == 'na',
    df['rating'] == 'low',
    df['rating'] == 'moderate',
    df['rating'] == 'considerable',
    df['rating'] == 'high',
    df['rating'] == 'extream',
)
_df['rating'] = np.select(conditions, values)

start = _df.iloc[0].name
max_end = _df.iloc[-1].name
end = start + window
slices = []
not_even_slice = []
one_day = np.timedelta64(1,'D')

while end <= max_end:
    df_slice = _df[(_df.index > start) & (_df.index <= end)]
    if len(df_slice) == 3:
        slices.append(df_slice)
        start = end
        end = start + window
    else:
        #not_even_slice.append(df_slice)
        start = start + one_day
        end = end + one_day
        

In [10]:
len(slices)

582

In [11]:
len(df) // 3

612

In [12]:
targets = slices[0]
for sample in slices[1:]:
    targets = pd.concat([targets, sample.iloc[-1:]])

In [13]:
targets.rating.value_counts()

2    292
1    180
3     90
4     20
0      2
Name: rating, dtype: int64

In [14]:
targets

,Snow Depth (cm) Start of Day Values,Precipitation Increment (mm),Air Temperature Average (degC),Wind Speed Average (km/hr),rating
date,,,,,
2008-12-18,43.733333,5.466667,-7.420000,11.900000,1
2008-12-19,41.533333,9.200000,-5.860000,15.966667,3
2008-12-20,44.200000,3.733333,-2.813333,6.400000,1
2008-12-23,65.666667,8.400000,-8.086667,7.300000,2
2008-12-26,105.133333,3.066667,-11.266667,3.700000,2
...,...,...,...,...,...
2020-12-20,61.333333,0.000000,2.680000,4.750000,2
2020-12-23,57.600000,0.200000,-3.013333,4.800000,1
2020-12-28,67.666667,3.600000,-5.406667,3.300000,3


In [15]:
dataset = []
for sample in slices:
    flat_sample = sample.to_numpy().flatten().tolist()
    dataset.append([*flat_sample[:-1], flat_sample[-1]])

In [16]:
df = pd.DataFrame(dataset)

In [17]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,43.733333,5.466667,-7.420000,11.900000,1.0,41.533333,9.200000,-5.860000,15.966667,3.0,44.200000,3.733333,-2.813333,6.400000,1.0
1,40.866667,14.266667,-0.206667,8.266667,1.0,50.733333,9.866667,-6.540000,12.433333,3.0,65.666667,8.400000,-8.086667,7.300000,2.0
2,65.200000,18.800000,-3.260000,10.200000,2.0,74.800000,44.600000,-7.293333,11.900000,3.0,105.133333,3.066667,-11.266667,3.700000,2.0
3,100.666667,3.400000,-2.993333,4.633333,1.0,91.933333,1.533333,0.900000,6.100000,1.0,88.933333,0.000000,2.020000,10.400000,1.0
4,77.000000,15.066667,-1.653333,12.466667,2.0,85.800000,0.000000,-9.320000,2.533333,2.0,83.266667,0.000000,-7.046667,3.966667,1.0


In [31]:
df = df[df[14] != 0]

In [32]:
df[14].value_counts()

2.0    292
1.0    179
3.0     89
4.0     20
Name: 14, dtype: int64

In [45]:
test_df = pd.DataFrame()
test_df = pd.concat([test_df, df[df[14]==1].sample(4)])
test_df = pd.concat([test_df, df[df[14]==2].sample(4)])
test_df = pd.concat([test_df, df[df[14]==3].sample(4)])
test_df = pd.concat([test_df, df[df[14]==4].sample(4)])

In [46]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
569,40.600000,0.000000,0.186667,3.300000,1.0,38.866667,0.333333,1.846667,9.250000,1.0,36.800000,0.400000,-1.000000,5.550000,1.0
509,286.066667,0.000000,-1.366667,2.250000,2.0,281.800000,1.533333,1.806667,4.250000,2.0,276.133333,8.200000,3.333333,7.550000,1.0
5,80.933333,2.666667,-1.006667,10.566667,1.0,80.466667,0.000000,2.342857,6.950000,2.0,77.600000,0.000000,4.493333,3.800000,1.0
223,131.266667,0.000000,-1.926667,6.466667,2.0,127.533333,0.200000,-4.700000,9.566667,2.0,128.066667,0.933333,-4.653333,5.033333,1.0
43,32.866667,1.000000,1.273333,3.533333,2.0,29.400000,0.000000,0.940000,1.766667,2.0,27.933333,0.000000,-1.073333,2.266667,2.0


In [49]:
test_df.index

Int64Index([569, 509, 5, 223, 43, 364, 567, 486, 324, 63, 37, 240, 401, 127,
            396, 387],
           dtype='int64')

In [50]:
train_df = df.drop(test_df.index)

In [52]:
train_df = train_df.reset_index()
test_df = test_df.reset_index()

In [53]:
# train_df.to_csv('./data/train.csv')
# test_df.to_csv('./data/test.csv')